In [2]:
import requests
from PIL import Image
from io import BytesIO
import pandas as pd
from transformers import CLIPProcessor, CLIPModel
import torch

import glob

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
df=pd.read_csv("/Users/kunkerdthaisong/cils/bangkok_traffy.csv")
df=df.dropna(subset=["type"])
df=df[df["type"].str.contains("ท่อ")] #30288 rows

In [13]:
df

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
5,2021-8Z8JQ3,"{ท่อระบายน้ำ,ถนน,ร้องเรียน,ความปลอดภัย,น้ำท่วม}",NaN,1) พื้นที่นี้ขาดหน่วยงานรับผิดชอบ เนื่องจากไม่...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.86190,13.72804",3/8 ขุมทอง-ลำต้อยติ่ง แขวง ขุมทอง เขตลาดกระบัง...,ขุมทอง,ลาดกระบัง,กรุงเทพมหานคร,2021-09-20 00:24:47.881831+00,กำลังดำเนินการ,NaN,0,2022-02-22 04:29:57.80173+00
9,2021-Q7XMCN,"{ความปลอดภัย,ท่อระบายน้ำ}",NaN,งานไม่เสร็จ ควรเก็บท่อHDPE ทั้งการวางการมัดท่อ...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.50779,13.79675",1 ถนน จรัญสนิทวงศ์ แขวง บางอ้อ เขตบางพลัด กรุง...,บางอ้อ,บางพลัด,กรุงเทพมหานคร,2021-09-24 04:15:40.312129+00,กำลังดำเนินการ,NaN,0,2022-02-22 04:29:46.037649+00
14,2021-F7A2PK,"{ความปลอดภัย,ท่อระบายน้ำ}",NaN,มีฝาท่อระบายน้ำชำรุดบริเวณ ซ.เพชรเกษม46 ช่วงตร...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.44755,13.71937",19/31 ซอย เพชรเกษม 46 แยก 11 แขวง บางหว้า เขตภ...,บางหว้า,ภาษีเจริญ,กรุงเทพมหานคร,2021-11-12 02:19:45.429843+00,กำลังดำเนินการ,NaN,0,2022-02-22 04:29:33.069509+00
38,2021-FUM99G,"{ทางเท้า,ท่อระบายน้ำ,สายไฟ}",NaN,"ทางเท้า ตามพิกัด มีฝาท่อเปิด และมีสายไฟ,เคเบิ้...",https://storage.googleapis.com/traffy_public_b...,NaN,"100.42688,13.78080",2 ถนน บรมราชชนนี แขวง ฉิมพลี เขตตลิ่งชัน กรุงเ...,ฉิมพลี,ตลิ่งชัน,กรุงเทพมหานคร,2021-12-10 10:09:44.281995+00,กำลังดำเนินการ,NaN,0,2022-02-22 04:28:21.869591+00
39,2021-4K7APR,"{ถนน,ทางเท้า,ท่อระบายน้ำ}",NaN,ฟุตบาทถนนเทพรักษ์ (ถนนเส้นตัดใหม่ตรงbig c สะพา...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.60634,13.89027",3/1703 ซ. 4 แยก 2 แขวง อนุสาวรีย์ เขตบางเขน กร...,อนุสาวรีย์,บางเขน,กรุงเทพมหานคร,2021-12-10 11:53:03.739874+00,กำลังดำเนินการ,NaN,0,2022-02-22 04:28:19.004857+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500149,2024-GCUPMP,"{ท่อระบายน้ำ,ถนน,ทางเท้า}","บริษัท โทรคมนาคมแห่งชาติ จำกัด (มหาชน),เขตบึงกุ่ม",ฝาท่อระบายน้ำชำรุดบนบาทวิถีทางเท้าฟุตปาธครับบร...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.64896,13.81050",140 351 แขวงนวมินทร์ เขตบึงกุ่ม กรุงเทพมหานคร ...,นวมินทร์,บึงกุ่ม,กรุงเทพมหานคร,2024-02-09 18:04:38.602911+00,กำลังดำเนินการ,NaN,0,2024-02-09 23:16:30.291194+00
500154,2024-686U3W,{ท่อระบายน้ำ},"ฝ่ายโยธา เขตจอมทอง,เขตจอมทอง",ท่ออุดตันมีน้ำขัง อาจเป็นแหล่งบ่อเกิดยุงลาย,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.47535,13.70117",32/63 Chumchon Sinlapa Det แขวง บางค้อ เขตจอมท...,บางค้อ,จอมทอง,กรุงเทพมหานคร,2024-02-09 18:29:25.367102+00,กำลังดำเนินการ,NaN,0,2024-02-10 00:30:01.56327+00
500173,2024-F2GGGL,"{ท่อระบายน้ำ,กีดขวาง}",เขตบางขุนเทียน,ตั้งร้านกีดขวางทางเดิน กีดขวางท่อระบายน้ำ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.44853,13.65287",65/6 ซอย อนามัยงานเจริญ แขวงท่าข้าม เขตบางขุนเ...,ท่าข้าม,บางขุนเทียน,กรุงเทพมหานคร,2024-02-10 00:16:48.696291+00,รอรับเรื่อง,NaN,0,2024-02-10 00:16:48.68544+00
500186,2024-GKLB7V,"{ท่อระบายน้ำ,ถนน,ร้องเรียน,น้ำท่วม}","ฝ่ายเทศกิจ เขตห้วยขวาง,เขตห้วยขวาง",ท่อระบายน้ำปากซอยพระรามเก้า13 หักจุดเชื่อมต่อท...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.58746,13.75414",769/9 ถนน พระราม 9 แขวง บางกะปิ เขตห้วยขวาง กร...,บางกะปิ,ห้วยขวาง,กรุงเทพมหานคร,2024-02-10 01:25:59.950318+00,รอรับเรื่อง,NaN,0,2024-02-10 02:02:11.617076+00


In [7]:
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

In [25]:
def run(image):
        inputs =processor(
            text=["manhole","manhole cover","sink hole","road","not road"], #text=["flood", "flooded road", "water on floor", "road", "no water","dried road"],
            images=image,
            return_tensors="pt",
            padding=True
        )
        with torch.no_grad():
            outputs =model(**inputs)
            logits_per_image = outputs.logits_per_image
            probs = logits_per_image.softmax(dim=1)
        return probs

In [30]:

#read image from url
def read_and_save_image(url, save_path):

    response = requests.get(url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        probs=run(image)
        if probs.squeeze(0)[:3].sum()>0.5 and probs.squeeze(0)[-1:].sum()<0.2: #probs.squeeze(0)[1] >0.3 and probs.squeeze(0)[:3].sum()>0.55 and probs.squeeze(0)[-2:].sum()<0.25:
            image.save(save_path)
            return probs
    else:
        print(f"Failed to download image from URL: {url}")
        

In [31]:
count=0
for i in df["photo"][0:]:
    save_path = f"/Users/kunkerdthaisong/cils/images2/{count}.jpg"
    read_and_save_image(i, save_path)
    count+=1

tensor([2.6878e-01, 2.9546e-01, 4.3248e-01, 3.5973e-04, 2.9184e-03])
tensor([2.6878e-01, 2.9546e-01, 4.3248e-01, 3.5973e-04, 2.9184e-03])
tensor([2.6878e-01, 2.9546e-01, 4.3248e-01, 3.5973e-04, 2.9184e-03])
tensor([2.6878e-01, 2.9546e-01, 4.3248e-01, 3.5973e-04, 2.9184e-03])
tensor([2.6878e-01, 2.9546e-01, 4.3248e-01, 3.5973e-04, 2.9184e-03])
tensor([2.6878e-01, 2.9546e-01, 4.3248e-01, 3.5973e-04, 2.9184e-03])
tensor([2.6878e-01, 2.9546e-01, 4.3248e-01, 3.5973e-04, 2.9184e-03])
tensor([2.6878e-01, 2.9546e-01, 4.3248e-01, 3.5973e-04, 2.9184e-03])
tensor([2.6878e-01, 2.9546e-01, 4.3248e-01, 3.5973e-04, 2.9184e-03])
tensor([2.6878e-01, 2.9546e-01, 4.3248e-01, 3.5973e-04, 2.9184e-03])
tensor([2.6878e-01, 2.9546e-01, 4.3248e-01, 3.5973e-04, 2.9184e-03])
tensor([2.6878e-01, 2.9546e-01, 4.3248e-01, 3.5973e-04, 2.9184e-03])


KeyboardInterrupt: 

In [68]:
count

3709

In [70]:
df

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
4,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00
5,2021-8Z8JQ3,"{ท่อระบายน้ำ,ถนน,ร้องเรียน,ความปลอดภัย,น้ำท่วม}",NaN,1) พื้นที่นี้ขาดหน่วยงานรับผิดชอบ เนื่องจากไม่...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.86190,13.72804",3/8 ขุมทอง-ลำต้อยติ่ง แขวง ขุมทอง เขตลาดกระบัง...,ขุมทอง,ลาดกระบัง,กรุงเทพมหานคร,2021-09-20 00:24:47.881831+00,กำลังดำเนินการ,NaN,0,2022-02-22 04:29:57.80173+00
7,2021-AFJYNF,"{ถนน,น้ำท่วม}",NaN,น้ำท่วม เอ่อล้นถนน น้ำระบายไม่ทัน บริเวณถ.พัฒน...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.71592,13.73614",3 ถนน พัฒนาชนบท 3 แขวง คลองสองต้นนุ่น เขตลาดกร...,คลองสองต้นนุ่น,ลาดกระบัง,กรุงเทพมหานคร,2021-09-22 05:06:09.308534+00,กำลังดำเนินการ,NaN,0,2022-02-22 04:29:51.833505+00
8,2021-HAJULK,"{น้ำท่วม,ถนน,ร้องเรียน}",NaN,มีการยกถนนในซอยเมื่อหลายปีที่แล้ว ทำให้บ้านอยู...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.71661,13.79225",95 ซอย 12 มีนบุรี กรุงเทพมหานคร 10510 ประเทศไทย,มีนบุรี,มีนบุรี,กรุงเทพมหานคร,2021-09-23 06:25:55.715532+00,กำลังดำเนินการ,NaN,0,2022-02-22 04:29:49.030483+00
12,2021-9U2NJT,{น้ำท่วม},"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53099,13.81853",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-10-14 10:45:27.713884+00,เสร็จสิ้น,NaN,0,2022-09-08 08:35:43.784519+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499881,2024-CGUG86,{น้ำท่วม},"ฝ่ายโยธา เขตจอมทอง,เขตจอมทอง,พนักงานกวาด เขตจอ...",น้ำท้วมขังไม่ระบายจนเกิดเป็นตะกอน ตะไคร่ ความส...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.47532,13.70119",32/63 Chumchon Sinlapa Det แขวง บางค้อ เขตจอมท...,บางค้อ,จอมทอง,กรุงเทพมหานคร,2024-02-09 11:36:29.046261+00,กำลังดำเนินการ,NaN,0,2024-02-10 00:33:18.127945+00
500019,2024-EU9VCX,"{ความสะอาด,น้ำท่วม,ถนน}",เขตหนองจอก,ท่อน้ำทิ้งตัน ทำให้น้ำเอ่อขึ้นถนน ทำให้ส่งกลิ่...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.81994,13.84463",102/5 ถ. เลียบวารี แขวงโคกแฝด เขตหนองจอก กรุงเ...,โคกแฝด,หนองจอก,กรุงเทพมหานคร,2024-02-09 14:23:03.422026+00,กำลังดำเนินการ,NaN,0,2024-02-09 14:44:40.527399+00
500049,2024-EV3TH7,{น้ำท่วม},"สำนักสิ่งแวดล้อม กทม.,เขตดุสิต",กทมนำถังน้ำมาให้ที่ชุมนุมเมื่อวันจันทร์ ยังไม่...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.51341,13.76294",พระอนุสาวรีย์พลเรือเอกสมเด็จพระเจ้าบรมวงศ์เธอ ...,สวนจิตรลดา,ดุสิต,กรุงเทพมหานคร,2024-02-09 14:56:20.800187+00,กำลังดำเนินการ,NaN,0,2024-02-10 01:40:37.04586+00
500186,2024-GKLB7V,"{ท่อระบายน้ำ,ถนน,ร้องเรียน,น้ำท่วม}","ฝ่ายเทศกิจ เขตห้วยขวาง,เขตห้วยขวาง",ท่อระบายน้ำปากซอยพระรามเก้า13 หักจุดเชื่อมต่อท...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.58746,13.75414",769/9 ถนน พระราม 9 แขวง บางกะปิ เขตห้วยขวาง กร...,บางกะปิ,ห้วยขวาง,กรุงเทพมหานคร,2024-02-10 01:25:59.950318+00,รอรับเรื่อง,NaN,0,2024-02-10 02:02:11.617076+00


In [73]:
img_list=glob.glob("/Users/kunkerdthaisong/cils/images/*.jpg")
len(img_list)

2379

In [90]:
num_list=[]
for i in img_list:
    num_list.append(int(i.split("/")[-1].split(".j")[0]))
    

In [94]:
num_list=sorted(num_list)

In [95]:
num_list

[0,
 1,
 2,
 4,
 5,
 7,
 11,
 15,
 18,
 19,
 20,
 22,
 23,
 24,
 25,
 35,
 45,
 47,
 52,
 53,
 57,
 60,
 63,
 65,
 66,
 67,
 73,
 74,
 75,
 76,
 78,
 81,
 85,
 90,
 92,
 96,
 101,
 105,
 108,
 111,
 112,
 115,
 116,
 117,
 119,
 120,
 121,
 124,
 130,
 131,
 132,
 134,
 136,
 137,
 138,
 141,
 142,
 143,
 146,
 147,
 150,
 153,
 154,
 157,
 158,
 159,
 165,
 168,
 169,
 171,
 172,
 175,
 179,
 183,
 185,
 186,
 188,
 202,
 203,
 209,
 212,
 219,
 223,
 225,
 226,
 227,
 229,
 230,
 231,
 233,
 236,
 237,
 239,
 240,
 244,
 249,
 253,
 254,
 257,
 265,
 266,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 281,
 284,
 286,
 290,
 292,
 293,
 300,
 312,
 313,
 317,
 321,
 326,
 328,
 336,
 337,
 338,
 339,
 340,
 342,
 344,
 345,
 346,
 349,
 350,
 351,
 352,
 353,
 357,
 361,
 366,
 377,
 379,
 384,
 385,
 387,
 388,
 389,
 391,
 392,
 393,
 397,
 400,
 401,
 404,
 406,
 407,
 409,
 422,
 424,
 428,
 434,
 437,
 439,
 444,
 445,
 448,
 450,
 454,
 455,
 459,
 460,
 467,
 472,
 473,
 474

In [100]:
df2=df.iloc[num_list]

In [107]:
df3=df2.reset_index()

In [112]:
df3.to_csv("/Users/kunkerdthaisong/cils/flooded_road.csv")

In [110]:
df3["photo"][3]

'https://storage.googleapis.com/traffy_public_bucket/attachment/2021-10/5a36fa060eb2a750e3ff47cdd30307f4639f445e.jpg'

In [111]:
response = requests.get(df3["photo"][3])
image = Image.open(BytesIO(response.content))
image.show()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
